This notebook reproduces https://journals.aps.org/pra/abstract/10.1103/PhysRevA.102.023526

In [1]:
using Revise, HarmonicBalance
include("../plotting.jl");

┌ Info: Precompiling HarmonicBalance [e13b9ff6-59c3-11ec-14b1-f3d2cc6c135e]
└ @ Base loading.jl:1423
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

┌ Warning: Package HarmonicBalance does not have Plots in its dependencies:
│ - If you have HarmonicBalance checked out for development and have
│   added Plots as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with HarmonicBalance
└ Loading Plots into HarmonicBalance from project dependency, future warnings for HarmonicBalance are suppressed.
┌ Warning: You are using Matplotlib 3.1.2, which is no longer
│ officialy supported by the Plots community. To ensure smooth Plots.jl
│ integration update your Matplotlib library to a version >= 3.4.0
│ 
│ If you have used Conda.jl to instal

# Define the differential equation of motion

In [2]:
@variables γ, F, α, ω0, F0, η, J, ω, t, θ, x(t), y(t);

# a vector of expressions - these must equal to zero
diff_eq = DifferentialEquation([d(x,t,2) + γ * d(x,t) + ω0^2 * x + α*x^3+ J*ω0*(x-y) - F0*cos(ω*t), 
            d(y,t,2) + γ * d(y,t) + ω0^2 * y + α*y^3 + J*ω0*(y-x) - η*F0*cos(ω*t)], [x,y])

# describe each variable using one or more Fourier components
add_harmonic!(diff_eq, x, ω)
add_harmonic!(diff_eq, y, ω)

harmonic_eqs = get_harmonic_equations(diff_eq);

# Solving the resulting polynomial equations

In [3]:
fixed_parameters = (
    ω0 => 1 , # natural frequency of separate modes
    γ => 1E-2,    # damping
    J => 3E-2,   # coupling term
    α => 1E-3,   # Kerr nonlinearity
    ω => 1. + 3E-2,  # pump frequency, resonant with antisymmetric mode
    η => -0.1,      # pumping leaking to site 2  (F2 = ηF1)
    F0 => 1      # pump amplitude (overriden in sweeps)
)

swept = F0 => LinRange(0.1, 0.35, 200)

res = get_steady_states(harmonic_eqs, swept, fixed_parameters, random_warmup=true)

Tracking 81 paths... 100%|██████████████████████████████| Time: 0:00:19
  # paths tracked:                  81
  # non-singular solutions (real):  11 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         11 (0)


A steady state result for 200 parameter points

Solution branches:   11
   of which real:    3
   of which stable:  2

Classes: stable, physical, Hopf, binary_labels


## Export a reference figure (naive steady states)

In [ ]:
default(linewidth=1.4, size=(380, 130), legend=false)
amps1 = real.(transform_solutions(res, "sqrt(u1^2 + v1^2)"))
amps2 = real.(transform_solutions(res, "sqrt(u2^2 + v2^2)"))

stable(amps, i) = replace(getindex.(res.classes["physical"], i) .* getindex.(amps,i) .* getindex.(res.classes["stable"], i), 0.0 => NaN)
unstable(amps, i) = replace(getindex.(res.classes["physical"], i) .* getindex.(amps,i) .* map(x -> !(x), getindex.(res.classes["stable"], i)), 0.0 => NaN)

x_axis = swept[2]

p1 = Plots.plot(x_axis, stable(amps1, 1), c=1,
    ylabel="\$  \\sqrt{u_1^2 + v_1^2} \$",
    ylims=[1.5,11], xlabel="\$ F\$")
Plots.plot!(x_axis, unstable(amps1, 1), style=:dashdotdot, c=1)
Plots.plot!(x_axis, stable(amps1, 2), c=2)
Plots.plot!(x_axis, unstable(amps1, 3), c=3, style=:dot)
annotate!((0.1, 0.9), text("(a)", 7, "Computer Modern"))

p2 = Plots.plot(x_axis, stable(amps2, 1), c=1,
    ylabel="\$  \\sqrt{u_2^2 + v_2^2} \$",
    ylims=[4,9.5], xlabel="\$ F\$")
Plots.plot!(x_axis, unstable(amps2, 1), style=:dashdotdot, c=1)
Plots.plot!(x_axis, stable(amps2, 2), c=2)
Plots.plot!(x_axis, unstable(amps2, 3), c=3, style=:dot)
annotate!((0.1, 0.9), text("(b)", 7, "Computer Modern"))

p3 = Plots.plot(p1, p2, size=(400,130))

In [ ]:
savefig(p3, dir * "limit_cycles/2_duffings_ref.svg")

# Time-dependent simulations

In [4]:
import HarmonicBalance.TimeEvolution: ODEProblem, DifferentialEquations.solve, ParameterSweep
fft_window(data) = HarmonicBalance.TimeEvolution.DSP.Windows.bartlett(length(data))

# first a sweep followed by free time-evolution
initial_state = res[1][1]
F0_lims = (0.1, 0.2)

T = 1E5
sweep = ParameterSweep(F0 => F0_lims, (0,T))
TDproblem = ODEProblem(harmonic_eqs, initial_state, sweep=sweep, timespan=(0,4*T))
TDsoln = solve(TDproblem, saveat=5/pi^2)

retcode: Success
Interpolation: 1st order linear
t: 789570-element Vector{Float64}:
      0.0
      0.5066059182116889
      1.0132118364233778
      1.5198177546350666
      2.0264236728467555
      2.5330295910584444
      3.0396355092701333
      3.546241427481822
      4.052847345693511
      4.5594532639051994
      5.066059182116888
      5.572665100328578
      6.0792710185402665
      ⋮
 399994.7555713847
 399995.26217730285
 399995.7687832211
 399996.2753891393
 399996.78199505754
 399997.2886009757
 399997.79520689393
 399998.30181281216
 399998.80841873033
 399999.31502464856
 399999.8216305668
 400000.0
u: 789570-element Vector{Vector{Float64}}:
 [1.3741941895950185, 2.6411947122587534, -3.537410766319138, -2.6069274018182558]
 [1.37419418984187, 2.6411947744709248, -3.5374107664992227, -2.6069274080337688]
 [1.374194190360748, 2.641194960785494, -3.537410767637986, -2.606927426626321]
 [1.3741941908212585, 2.6411952707157242, -3.5374107706297653, -2.606927457516264]
 [1.37

In [ ]:
# FT it and show spectrum 

t1 = Int(round(0.5*length(TDsoln.t)))

ts = TDsoln.t[t1:end]
ω_drive = 1.0003

x_lab = getindex.(TDsoln.u,1)[t1:end] .* cos.(ts*ω_drive) + getindex.(TDsoln.u,2)[t1:end] .* sin.(ts*ω_drive);
y_lab = getindex.(TDsoln.u,3)[t1:end] .* cos.(ts*ω_drive) + getindex.(TDsoln.u,4)[t1:end] .* sin.(ts*ω_drive);

fft_x = HarmonicBalance.FFT(x_lab, ts, window=fft_window(x_lab));
fft_y = HarmonicBalance.FFT(y_lab, ts, window=fft_window(y_lab));

default(xlabel="\$ \\Omega \$")

p3 = Plots.plot(fft_x[2], abs.(fft_x[1][1]), xlim=(0.95,1.05),top_margin=-1mm, ylabel="FT\$\\,(x)\\, [\\Omega ]\$" )
annotate!((0.12, 0.9), text("(c)", 7, "Computer Modern"))

p4 = Plots.plot(fft_y[2], abs.(fft_y[1][1]), xlim=(0.95,1.05),top_margin=-1mm, c=2, ylabel="FT\$\\,(y)\\, [\\Omega ]\$" )
annotate!((0.12, 0.9), text("(d)", 7, "Computer Modern"))
Plots.plot(p3, p4, size=(800,400))

In [ ]:
x_axis = TDsoln.t
y_axis = transform_solutions(TDsoln, "sqrt(u1^2 + v1^2)", harmonic_eqs);
p1 = Plots.plot(x_axis .* 1E-5, y_axis, xlabel="\$T  \\: ×\\: 10^{-5} \$", 
    ylabel="\$  \\sqrt{u_1^2 + v_1^2} \$", size=(160, 100))
Plots.plot!(twinx(), x_axis .* 1E-5, sweep[F0].(TDsoln.t), c=:black, style=:dash, xlabel="", ylabel="\$ F \$",
yticks=[0.1,0.15,0.2])
annotate!((0.12, 0.9), text("(a)", 7, "Computer Modern"))

t1 = Int(round(0.5*length(TDsoln.t)));
u1s = getindex.(TDsoln.u, 1)[t1:end]
v1s = getindex.(TDsoln.u, 2)[t1:end]
u2s = getindex.(TDsoln.u, 3)[t1:end]
v2s = getindex.(TDsoln.u, 4)[t1:end];

p2 = Plots.plot(u1s, v1s, xlabel="\$ u \$", ylabel=" \$ v \$", xticks=[-6,-4,-2,0,2])
Plots.plot!(u2s, v2s)
annotate!((0.12, 0.9), text("(b)", 7, "Computer Modern"))

Plots.plot(p1, p2, size=(800,400))

In [ ]:
# time-dependent sweep

T = 2E5

U0_forw = res[1][1]
U0_back = res[180][1]
F0_start = substitute_all(F0, U0_forw)
F0_fin = substitute_all(F0, U0_back)

sweep_forw = ParameterSweep(F0 => (F0_start, F0_fin), (0,T))
TDproblem = ODEProblem(harmonic_eqs, U0_forw, sweep=sweep_forw, timespan=(0,T), W=3E-6)
TDsoln_forw = solve(TDproblem, HarmonicBalance.TimeEvolution.RandomEM(), dt=0.5);

sweep_back = ParameterSweep(F0 => (F0_fin, F0_start), (0,T))
TDproblem = ODEProblem(harmonic_eqs, U0_back, sweep=sweep_back, timespan=(0,T), W=3E-6)
TDsoln_back = solve(TDproblem, HarmonicBalance.TimeEvolution.RandomEM(), dt=0.5);

x_axis = TDsoln_forw.t

y_forw = sqrt.(getindex.(TDsoln_forw.u, 1) .^2  + getindex.(TDsoln_forw.u, 2) .^2)
y_back = reverse(sqrt.(getindex.(TDsoln_back.u, 1) .^2  + getindex.(TDsoln_back.u, 2) .^2))

In [ ]:

x_axis = Float64.(sweep_forw[F0].(TDsoln_forw.t))
p5 = Plots.plot(x_axis, y_forw, top_margin=-2mm, xlabel="\$F\$", 
    ylabel="\$  \\sqrt{u_1^2 + v_1^2} \$", right_margin=3.5mm, size=(160, 100), opacity=0.5)
Plots.plot!(x_axis, y_back, c=:red, opacity=0.5);
annotate!((0.06, 0.9), text("(e)", 7, "Computer Modern"))
quiver!((0.11, 3.5), quiver=([0.02], [-0.2]), c=1)
quiver!((0.32, 3.1), quiver=([-0.02], [-0.6]), c=:red, opacity=0.7);
Plots.plot(p5, size=(600,400))

In [ ]:
l = @layout [ a b ; c d ; e ]

Plots.plot(p1, p2, p3, p4, p5, layout=l, size=(390, 350), bottom_margin=-2mm, left_margin=5mm)

In [ ]:
savefig(dir * "limit_cycles/2_duffings_timedep.svg")

In [ ]:
using NumericalIntegration
norm = ts[end] - ts[1]
indices =  0.975 .< fft[2] .< 0.99
integrate(fft[2][indices], abs.(fft[1][1])[indices]) * norm / (2pi)